In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
from google.colab import drive
drive.mount("/content/drive")

Mounted at /content/drive


In [2]:
df=pd.read_csv('/content/drive/MyDrive/data/appe_no_dates.csv')

In [3]:
df.head()

,Open,High,Low,Close,Volume
0,142.90,144.75,142.90,144.18,19201712
1,143.02,143.50,142.41,142.73,24128782
2,143.69,144.79,142.72,144.09,21569557
3,144.88,145.30,143.10,143.50,14277848
4,144.45,144.96,143.78,144.02,23024107


In [5]:
df.shape


(26, 5)

In [4]:
rng = pd.date_range(start="3/1/2022", end="04/05/2022", freq='B')
rng

DatetimeIndex(['2022-03-01', '2022-03-02', '2022-03-03', '2022-03-04',
               '2022-03-07', '2022-03-08', '2022-03-09', '2022-03-10',
               '2022-03-11', '2022-03-14', '2022-03-15', '2022-03-16',
               '2022-03-17', '2022-03-18', '2022-03-21', '2022-03-22',
               '2022-03-23', '2022-03-24', '2022-03-25', '2022-03-28',
               '2022-03-29', '2022-03-30', '2022-03-31', '2022-04-01',
               '2022-04-04', '2022-04-05'],
              dtype='datetime64[ns]', freq='B')

Using 'B' frequency is not going to help because 4th July was holiday and 'B' is not taking that into account. It only accounts for weekends

**Using CustomBusinessDay to generate US holidays calendar frequency**

In [6]:
from pandas.tseries.holiday import USFederalHolidayCalendar
from pandas.tseries.offsets import CustomBusinessDay

In [9]:
us_cal = CustomBusinessDay(calendar=USFederalHolidayCalendar())

rng = pd.date_range(start="3/1/2022", end="04/05/2022", freq=us_cal)
rng

DatetimeIndex(['2022-03-01', '2022-03-02', '2022-03-03', '2022-03-04',
               '2022-03-07', '2022-03-08', '2022-03-09', '2022-03-10',
               '2022-03-11', '2022-03-14', '2022-03-15', '2022-03-16',
               '2022-03-17', '2022-03-18', '2022-03-21', '2022-03-22',
               '2022-03-23', '2022-03-24', '2022-03-25', '2022-03-28',
               '2022-03-29', '2022-03-30', '2022-03-31', '2022-04-01',
               '2022-04-04', '2022-04-05'],
              dtype='datetime64[ns]', freq='C')

In [10]:
df.set_index(rng,inplace=True)
df.head()

,Open,High,Low,Close,Volume
2022-03-01,142.90,144.75,142.90,144.18,19201712
2022-03-02,143.02,143.50,142.41,142.73,24128782
2022-03-03,143.69,144.79,142.72,144.09,21569557
2022-03-04,144.88,145.30,143.10,143.50,14277848
2022-03-07,144.45,144.96,143.78,144.02,23024107


You can define your own calendar using AbstractHolidayCalendar as shown below. USFederalHolidayCalendar is the only calendar available in pandas library and it serves as an example for those who want to write their own custom calendars. Here is the link for USFederalHolidayCalendar implementation https://github.com/pandas-dev/pandas/blob/master/pandas/tseries/holiday.py

**AbstractHolidayCalendar**

In [13]:
from pandas.tseries.holiday import AbstractHolidayCalendar, nearest_workday, Holiday
class myCalendar(AbstractHolidayCalendar):
    rules = [
        Holiday('My Birth Day', month=3, day=1),#, observance=nearest_workday),
    ]
    
my_bday = CustomBusinessDay(calendar=myCalendar())
pd.date_range(start="3/1/2022", end="04/05/2022",freq=my_bday)

DatetimeIndex(['2022-03-02', '2022-03-03', '2022-03-04', '2022-03-07',
               '2022-03-08', '2022-03-09', '2022-03-10', '2022-03-11',
               '2022-03-14', '2022-03-15', '2022-03-16', '2022-03-17',
               '2022-03-18', '2022-03-21', '2022-03-22', '2022-03-23',
               '2022-03-24', '2022-03-25', '2022-03-28', '2022-03-29',
               '2022-03-30', '2022-03-31', '2022-04-01', '2022-04-04',
               '2022-04-05'],
              dtype='datetime64[ns]', freq='C')

**CustomBusinessDay**

Weekend in egypt is Friday and Saturday. Sunday is just a normal weekday and you can handle this custom week schedule using CystomBysinessDay with weekmask as shown below

In [14]:
egypt_weekdays = "Sun Mon Tue Wed Thu"

b = CustomBusinessDay(weekmask=egypt_weekdays)

pd.date_range(start="01/1/2022",periods=20,freq=b)

DatetimeIndex(['2022-01-02', '2022-01-03', '2022-01-04', '2022-01-05',
               '2022-01-06', '2022-01-09', '2022-01-10', '2022-01-11',
               '2022-01-12', '2022-01-13', '2022-01-16', '2022-01-17',
               '2022-01-18', '2022-01-19', '2022-01-20', '2022-01-23',
               '2022-01-24', '2022-01-25', '2022-01-26', '2022-01-27'],
              dtype='datetime64[ns]', freq='C')

**You can also add holidays to this custom business day frequency**

In [15]:
b = CustomBusinessDay(holidays=['2022-03-19', '2022-03-26'], weekmask=egypt_weekdays)

pd.date_range(start="3/01/2022",periods=30,freq=b)

DatetimeIndex(['2022-03-01', '2022-03-02', '2022-03-03', '2022-03-06',
               '2022-03-07', '2022-03-08', '2022-03-09', '2022-03-10',
               '2022-03-13', '2022-03-14', '2022-03-15', '2022-03-16',
               '2022-03-17', '2022-03-20', '2022-03-21', '2022-03-22',
               '2022-03-23', '2022-03-24', '2022-03-27', '2022-03-28',
               '2022-03-29', '2022-03-30', '2022-03-31', '2022-04-03',
               '2022-04-04', '2022-04-05', '2022-04-06', '2022-04-07',
               '2022-04-10', '2022-04-11'],
              dtype='datetime64[ns]', freq='C')